In [3]:
import numpy as np
import mne # library for eeg processing


# Prepare dataset

In [4]:
# subject_list_valid = [1,2,3,4,5,8,9,10,11,12,13,14,15,16,18,20,21,22,23,24,26]
subject_list_valid = [1,2]

eeg = {}
epochs = {}
for i in subject_list_valid:
    dataPath = './Dataset-2-EEG_images/'
    datafilename =  dataPath + str(i)+'-P-cleaned.fif'
    print(i)
    eeg[i] = mne.read_epochs(datafilename)
    epochs[i] = eeg[i].get_data()

1
Reading /Users/a0/Downloads/Dataset-2-EEG_images/1-P-cleaned.fif ...
    Found the data of interest:
        t =       0.00 ...    1496.09 ms
        0 CTF compensation matrices available


<ipython-input-4-49edbe72ae26>:10: RuntimeWarning: This filename (./Dataset-2-EEG_images/1-P-cleaned.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  eeg[i] = mne.read_epochs(datafilename)


Not setting metadata
Not setting metadata
371 matching events found
No baseline correction applied
0 projection items activated
2
Reading /Users/a0/Downloads/Dataset-2-EEG_images/2-P-cleaned.fif ...


<ipython-input-4-49edbe72ae26>:10: RuntimeWarning: This filename (./Dataset-2-EEG_images/2-P-cleaned.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  eeg[i] = mne.read_epochs(datafilename)


    Found the data of interest:
        t =       0.00 ...    1496.09 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
880 matching events found
No baseline correction applied
0 projection items activated


In [5]:
for i in subject_list_valid:
    print('Object'+str(i)+':')
    print('Number of trials = %d\nNumber of channels = %d\nNumber of time points = %d' %(epochs[i].shape)) 


Object1:
Number of trials = 371
Number of channels = 64
Number of time points = 384
Object2:
Number of trials = 880
Number of channels = 64
Number of time points = 384


In [6]:
for i in subject_list_valid:
    print('Object'+str(i)+':')
    print('Number of trials in FN condition: ', eeg[i]['FN'].get_data().shape[0])
    print('Number of trials in FP condition: ', eeg[i]['FP'].get_data().shape[0])
    print('Number of trials in FU condition: ', eeg[i]['FU'].get_data().shape[0])

    print('Number of trials in NN condition: ', eeg[i]['NN'].get_data().shape[0])
    print('Number of trials in NP condition: ', eeg[i]['NP'].get_data().shape[0])
    print('Number of trials in NU condition: ', eeg[i]['NU'].get_data().shape[0])

Object1:
Number of trials in FN condition:  99
Number of trials in FP condition:  104
Number of trials in FU condition:  94
Number of trials in NN condition:  30
Number of trials in NP condition:  27
Number of trials in NU condition:  17
Object2:
Number of trials in FN condition:  225
Number of trials in FP condition:  234
Number of trials in FU condition:  227
Number of trials in NN condition:  64
Number of trials in NP condition:  63
Number of trials in NU condition:  67


In [7]:
familiar_all = {}
novel_all = {}
for i in subject_list_valid:
    print('Object'+str(i)+':')
    familiar_all[i] = eeg[i]['FU', 'FP', 'FN']
    print('Number of trials in all FAMILIAR conditions: ', familiar_all[i].get_data().shape[0])
    novel_all[i] = eeg[i]['NU', 'NP', 'NN']
    print('Number of trials in all NOVEL conditions: ', novel_all[i].get_data().shape[0])

Object1:
Number of trials in all FAMILIAR conditions:  297
Number of trials in all NOVEL conditions:  74
Object2:
Number of trials in all FAMILIAR conditions:  686
Number of trials in all NOVEL conditions:  194


In [8]:
conc_familiar = familiar_all[1].get_data()
for i in subject_list_valid[1:]:
    conc_familiar = np.concatenate((conc_familiar, familiar_all[i].get_data()), axis=0)

conc_novel = novel_all[1].get_data()
for i in subject_list_valid[1:]:
    conc_novel = np.concatenate((conc_novel, novel_all[i].get_data()), axis=0)

In [9]:
del familiar_all
del novel_all

In [10]:
conc_familiar.shape

(983, 64, 384)

In [11]:
conc_novel.shape

(268, 64, 384)

In [12]:
avg_familiar = conc_familiar.mean(axis=1)
avg_novel = conc_novel.mean(axis=1)

In [13]:
avg_familiar.shape, avg_novel.shape

((983, 384), (268, 384))

In [14]:
# flat_familiar = conc_familiar.reshape(-1,64*384)
# flat_novel = conc_novel.reshape(-1,64*384)

In [15]:
# flat_familiar.shape, flat_novel.shape

# Dynamic Time Warping


In [19]:
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [20]:
DTWDistance(epochs[1][1][1], epochs[2][1][1], 10)

150.1247395994242

In [21]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return np.sqrt(LB_sum)

# KNN

In [23]:
from sklearn.metrics import classification_report

def knn(train,test,w):
    preds=[]
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    return classification_report(test[:,-1],preds)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
alldata = np.concatenate((avg_familiar, avg_novel), axis=0)

In [28]:
avg_familiar.shape, avg_novel.shape

((983, 384), (268, 384))

In [29]:
alldata.shape

(1251, 384)

In [30]:
labels = np.concatenate((np.ones(avg_familiar.shape[0]), np.zeros(avg_novel.shape[0])))

In [31]:
labels.shape

(1251,)

In [32]:
del eeg
del epochs

In [33]:
X_train, X_test, y_train, y_test = train_test_split(alldata, labels, test_size=0.33, random_state=42)

In [34]:
train = np.c_[ X_train, y_train ]  
test = np.c_[ X_test, y_test ]  

In [36]:
print(knn(train,test,4))

              precision    recall  f1-score   support

         0.0       0.24      0.24      0.24        91
         1.0       0.79      0.78      0.78       322

    accuracy                           0.66       413
   macro avg       0.51      0.51      0.51       413
weighted avg       0.66      0.66      0.66       413



In [37]:
train.shape

(838, 385)